In [ ]:
from keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
from keras.layers import Input
from keras import backend as K

K.reset_uids()

model = InceptionV3(weights='imagenet')
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
img_path = 'Gazelle/Gazelle.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])
orig_acc = decode_predictions(preds, top=3)[0][0][2]
print('Prob:', orig_acc)

Predicted: [('n02423022', 'gazelle', 0.9439806), ('n02422699', 'impala', 0.004907405), ('n02690373', 'airliner', 0.0010229984)]
Prob: 0.9439806


# This part to compute the difference in prediction probability when deleting each filter for every last convolution layer. The last convolution layers are concatinated into mixed10 

In [ ]:
from kerassurgeon.operations import delete_layer, insert_layer, delete_channels, replace_layer
from kerassurgeon import Surgeon
from kerassurgeon import identify
from kerassurgeon import utils
from keras import backend as K
from keras.models import Sequential, Model, load_model

filterTotal=0

arryOfDictFiltByLayer = []
convLayers =[]
for layer in model.layers:  
    if 'conv2d' in layer.name:
        convLayers.append(layer)
convLayersDelete =   convLayers[87: 94]

for layer in convLayersDelete:
  if layer.name=='conv2d_91':
    print('--------------------')
    print('layer', layer.name)
    print('--------------------')
    filters= layer.get_weights()[0]
    num_filters_Layer = len (filters[0, 0, 0, :])
    weights_dic = {}
    for j in range (num_filters_Layer):
        filterTotal=filterTotal + 1
        print ('filt{}'.format (j))
        filt = j

        print('layer name:', layer.name)
        channel = [j]
        new_model = delete_channels (model, layer, channel, copy=True)
        new_model.compile (optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])
        preds = new_model.predict(x)
        New_accu = decode_predictions(preds, top=3)[0][0][2]
        print ('acc after purning (%.2f%%)', New_accu)
        print('difference: ', (orig_acc - New_accu))
        tuple = orig_acc - New_accu, 1
        weights_dic[filt] = tuple

        K.clear_session ()
        K.reset_uids()
        model = InceptionV3(weights='imagenet')

        print("Model created")

    weights_dic_sort = sorted (weights_dic.items (), key=lambda kv: kv[1], reverse=True)
    print ('Filters importance {}\n'.format (1), weights_dic_sort)
    arryOfDictFiltByLayer.append(weights_dic_sort)
    print('filter total number', filterTotal)

# Concerned layers
# --------------------
# layer conv2d_93
# --------------------
# --------------------
# layer conv2d_87
# --------------------
# --------------------
# layer conv2d_88
# --------------------
# --------------------
# layer conv2d_91
# --------------------
# --------------------
# layer conv2d_92
# --------------------
# --------------------
# layer conv2d_85
# --------------------

# conv2d_87 (Conv2D)              (None, 8, 8, 384)    442368      activation_86[0][0]              
# __________________________________________________________________________________________________
# conv2d_88 (Conv2D)              (None, 8, 8, 384)    442368      activation_86[0][0]              
# __________________________________________________________________________________________________
# conv2d_91 (Conv2D)              (None, 8, 8, 384)    442368      activation_90[0][0]              
# __________________________________________________________________________________________________
# conv2d_92 (Conv2D)              (None, 8, 8, 384)    442368      activation_90[0][0]
# __________________________________________________________________________________________________
# conv2d_85 (Conv2D)              (None, 8, 8, 320)    655360      mixed9[0][0]                     
# __________________________________________________________________________________________________
# conv2d_93 (Conv2D)              (None, 8, 8, 192)    393216      average_pooling2d_8[0][0]    


--------------------
layer conv2d_91
--------------------
filt0
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.9440554
difference:  -7.480383e-05
Model created
filt1
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.9439815
difference:  -9.536743e-07
Model created
filt2
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.9437271
difference:  0.00025349855
Model created
filt3
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.94398504
difference:  -4.4703484e-06
Model created
filt4
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


...

acc after purning (%.2f%%) 0.9439634
difference:  1.7166138e-05
Model created
filt380
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.94400066
difference:  -2.0086765e-05
Model created
filt381
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.9440127
difference:  -3.2126904e-05
Model created
filt382
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.94282794
difference:  0.0011526346
Model created
filt383
layer name: conv2d_91
Deleting 1/384 channels from layer: conv2d_91


acc after purning (%.2f%%) 0.94401854
difference:  -3.796816e-05
Model created
Resp and loss for conv layer 1
 [(202, (0.005214393, 1)), (366, (0.005087912, 1)), (227, (0.0049285293, 1)), (223, (0.0047789216, 1)), (314, (0.0047417283, 1)), (377, (0.004383385, 1)), (245, (0.0043103695, 1)), (306, (0.0037683845, 1)), (80, (0.003584981, 1)), (246, (0.0035830736, 1)), (255, (0.0034936666, 1)), (369, (0.003367424, 1)), (217, (0.0030900836, 1)), (262, (0.0026903749, 1)), (284, (0.0026341677, 1)), (54, (0.0023847222, 1)), (62, (0.002370119, 1)), (4, (0.0021852255, 1)), (115, (0.0021741986, 1)), (93, (0.002125442, 1)), (204, (0.0020558834, 1)), (188, (0.0020474195, 1)), (330, (0.002044797, 1)), (11, (0.0020152926, 1)), (201, (0.0018855929, 1)), (360, (0.0018830299, 1)), (338, (0.0017622113, 1)), (309, (0.001753509, 1)), (239, (0.0017294884, 1)), (230, (0.0015907884, 1)), (275, (0.0015202165, 1)), (210, (0.001488626, 1)), (375, (0.0014448166, 1)), (88, (0.0013309121, 1)), (288, (0.0012187362, 1